In [6]:
import sys
import pandas as pd
from pathlib import Path
if '..' not in sys.path:
    sys.path.append('..')
    
import dataset

In [79]:
submits = []
for path in sorted(Path('../../_data/voting/').glob('*.csv')):
    print(path.name)
    submits.append(pd.read_csv(path, index_col=0))

07-20170720-232049-b7-albu-0.93374-03-albu-mean3_0.933493-bayes.csv
20170720-005737-b6-albu-0.93359-bayes-test.csv
20170720-005737-b6-albu-0.93359.csv
20170720-005737-b6-albu-0.93359_weather_hack0606.csv
20170720-232049-b7-albu-0.93374.csv
20170721-010132-b8-albu-0.93373-weather-hack-02.csv


In [80]:
scores = [0.93398, 0.93396, 0.93392, 0.93401, 0.93392, 0.93408]

In [81]:
submits[0][:3]

,tags
image_name,
test_0,clear primary
test_1,clear primary
test_2,partly_cloudy primary


In [14]:
def to_flat(df):
    flat = pd.DataFrame(0, index=df.index, columns=dataset.CLASSES)
    for i, tags in enumerate(df['tags']):
        row = flat.iloc[i]
        for cls in tags.split(' '):
            row[cls] = 1
    return flat

In [82]:
flat_submits = [to_flat(x) for x in submits]

In [83]:
mean = sum(flat_submits) / len(flat_submits)
mean[:3]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
image_name,,,,,,,,,,,,,,,,,
test_0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
test_1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
test_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [84]:
submits[0][:3]

,tags
image_name,
test_0,clear primary
test_1,clear primary
test_2,partly_cloudy primary


In [85]:
((mean != 0) & (mean != 1)).any(1).sum()

3308

In [86]:
vote_2 = mean >= (2/5 - 1e-5)
vote_3 = mean >= (3/5 - 1e-5)

In [87]:
(vote_2 != vote_3).any(1).sum()

371

In [88]:
skip = {2, 4}
w_mean = (sum(df * w for i, (df, w) in enumerate(zip(flat_submits, scores)) if i not in skip) /
          sum(s for i, s in enumerate(scores) if i not in skip))
vote_fair = w_mean > 0.5

In [89]:
(vote_2 != vote_fair).any(1).sum()

391

In [90]:
(vote_3 != vote_fair).any(1).sum()

504

In [91]:
vote_2.sum()

agriculture          23960
artisinal_mine         616
bare_ground           2048
blooming               525
blow_down              116
clear                45981
cloudy                4948
conventional_mine      142
cultivation          10332
habitation            7332
haze                  5698
partly_cloudy        12776
primary              58372
road                 14896
selective_logging      663
slash_burn             312
water                13769
dtype: int64

In [92]:
vote_3.sum()

agriculture          23928
artisinal_mine         613
bare_ground           2033
blooming               520
blow_down              114
clear                45954
cloudy                4930
conventional_mine      141
cultivation          10295
habitation            7308
haze                  5674
partly_cloudy        12730
primary              58344
road                 14841
selective_logging      658
slash_burn             301
water                13721
dtype: int64

In [93]:
vote_fair.sum()

agriculture          23919
artisinal_mine         614
bare_ground           2037
blooming               522
blow_down              116
clear                45938
cloudy                4934
conventional_mine      142
cultivation          10333
habitation            7330
haze                  5631
partly_cloudy        12711
primary              58233
road                 14890
selective_logging      660
slash_burn             309
water                13715
dtype: int64

In [95]:
w_mean[((w_mean > 0.5) & (w_mean < 0.7)).any(1)][:10]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
image_name,,,,,,,,,,,,,,,,,
test_6,1.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,1.00000,1.00000,0.0,1.00000,1.0,0.000000,0.00000,0.0,0.50002
test_101,1.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.00000,1.00000,0.0,0.50002,1.0,1.000000,0.00000,0.0,0.00000
test_178,0.0,0.0,0.0,0.0,0.0,0.0,0.50002,0.0,0.00000,0.00000,1.0,0.00000,1.0,0.000000,0.00000,0.0,0.00000
test_193,0.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.00000,0.00000,0.0,0.00000,1.0,0.250019,0.50002,0.0,1.00000
test_430,1.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,1.00000,0.50002,0.0,0.00000,1.0,0.000000,0.00000,0.0,0.00000
test_457,1.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0,1.00000,1.0,0.500020,0.00000,0.0,0.00000
test_508,1.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.00000,1.00000,0.0,0.00000,1.0,0.500020,0.00000,0.0,1.00000
test_591,1.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.00000,0.50002,0.0,0.00000,1.0,1.000000,0.00000,0.0,0.00000
test_606,1.0,0.0,0.0,0.0,0.0,1.0,0.00000,0.0,0.00000,0.50002,0.0,0.00000,1.0,1.000000,0.00000,0.0,0.00000


In [96]:
w_mean[(vote_3 != vote_fair).any(1)][:10]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
image_name,,,,,,,,,,,,,,,,,
test_6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.00000,1.0,0.0,1.00000,1.000000,0.00000,0.0,0.0,0.50002
test_34,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00000,0.0,0.0,0.00000,0.000000,0.00000,0.0,0.0,0.49998
test_101,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00000,1.0,0.0,0.50002,1.000000,1.00000,0.0,0.0,0.00000
test_208,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.00000,0.0,0.0,0.00000,0.499993,0.00000,0.0,0.0,1.00000
test_621,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.50002,0.0,0.0,0.00000,1.000000,0.00000,0.0,0.0,0.00000
test_668,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.00000,1.0,0.0,0.00000,1.000000,0.50002,0.0,0.0,1.00000
test_714,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00000,0.0,0.0,1.00000,0.499980,0.00000,0.0,0.0,0.00000
test_1056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,1.0,0.50002,1.000000,0.00000,0.0,0.0,0.00000
test_1084,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00000,0.0,0.0,0.50002,1.000000,0.00000,0.0,0.0,0.00000


In [97]:
submit = vote_fair
out_df = pd.DataFrame([
    {'tags': ' '.join(c for c in submit.columns if row[c])
     } for image_name, row in submit.iterrows()],
    index=submit.index)

In [98]:
out_df.to_csv('../../vote_fair.csv', index_label='image_name')
! head ../../vote_fair.csv

image_name,tags
test_0,clear primary
test_1,clear primary
test_2,partly_cloudy primary
test_3,agriculture clear cultivation primary
test_4,cloudy partly_cloudy primary
test_5,clear primary
test_6,agriculture clear cultivation habitation partly_cloudy primary water
test_7,agriculture clear habitation primary road
test_8,clear primary
